# Install Packages

In [ ]:
!pip install --upgrade google-cloud-bigquery[pandas]

# Connect to Google BigQuery

In [ ]:
from google.cloud import bigquery

# Set the parameters below
project = "<my-project>"
location = "<my-location>"

dataset_name = "<my-bq-dataset>"
raw_table_name = "<my-raw-table>"
avg_table_name = "<my-average-table>"

# Set the device clientid to be visualized 
client_name = "<my-client-id>"

# Create client connection
client = bigquery.Client(project=project)
# Fetch the table objects
raw_table_ref = client.dataset(dataset_name).table(raw_table_name)
avg_table_ref = client.dataset(dataset_name).table(avg_table_name)

raw_table = client.get_table(raw_table_ref)
avg_table = client.get_table(avg_table_ref)
tables = [raw_table, avg_table]

# View table schema

In [ ]:
from IPython.display import HTML, display
import tabulate

# Display the table schemas as tables
for table in tables:
    print("\nTable: {:15}   {} rows".format(table.table_id, table.num_rows))
    tab = []
    for schema in table.schema:
        row = [schema.name, schema.field_type]
        tab.append(row)
        
    display(HTML(tabulate.tabulate(tab, tablefmt='html')))

# Query most recent rows

In [ ]:
query = ("""
    SELECT
      *
    FROM
      `%s.%s.%s`
    WHERE
      clientid = '%s'
    ORDER BY
      timestamp DESC
    LIMIT
      5
    """ %(project, dataset_name, avg_table_name, client_name))

# Run the query on BigQuery
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location=location)  # API request - starts the query

# Display the rows in a tabulated format
rows = []
for row in query_job:  # API request - fetches results
    # Row values can be accessed by field name or index
    r = [row.timestamp, row.clientid, row.temperature, row.pressure]
    rows.append(r)
    
display(HTML(tabulate.tabulate(rows, tablefmt='html')))

And the average table has 1 row per minute, showing the last 5 minutes' sliding window average values

# Query: Average temperature values for the last 24 hours

## Import maptplotlib graphing library and make a SQL query

In [ ]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

query = ("""
SELECT
  timestamp, clientid, temperature, pressure
FROM
  `%s.%s.%s`
WHERE
  clientid = "%s"
ORDER BY
  timestamp DESC
LIMIT
  1440
    """ %(project, dataset_name, avg_table_name, client_name))

# Run query to fetch metrics from the average table
query_job = client.query(query, location=location)

datetimes = []
temperature_avgs = []
pressure_avgs = []

for row in query_job:
    datetimes.append(row['timestamp'])
    temperature_avgs.append(row['temperature'])
    pressure_avgs.append(row['pressure'])

## Create a simple line chart to plot the average temperature values

In [ ]:
# Create a line chart for temperature
plt.subplot(2,1,1)
plt.plot(datetimes, temperature_avgs)
plt.gcf().autofmt_xdate()
plt.xlabel('DateTime')
plt.ylabel('Temp Avg')

# Create a line chart for pressure
plt.subplot(2,1,2)
plt.plot(datetimes, pressure_avgs)
plt.gcf().autofmt_xdate()
plt.xlabel('DateTime')
plt.ylabel('Pressure Avg')

plt.show()

## Create a scatter chart to show the correlation between temperature and pressure values

In [ ]:
plt.scatter(temperature_avgs, pressure_avgs)
plt.show()